**OpenAI API Key Setup**

In this notebook, we ensure secure access to OpenAI services by configuring the API key using Google Colab’s secret management. The code imports necessary modules and retrieves the OPENAI_API_KEY from Colab’s encrypted storage, setting it as an environment variable

In [26]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

**AI Learning Tutor Initialization**

This setup imports key libraries and configures the AI tutor with OpenAI’s gpt-3.5-turbo model and embedding tools. By combining Gradio, CrewAI, and LangChain components, the foundation is laid for interactive and intelligent lesson generation, enabling dynamic and context-aware learning experiences.


In [27]:
import gradio as gr
import os
from crewai import Agent, Crew, Task
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Setup OpenAI LLM and Embeddings
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
embedding_model = OpenAIEmbeddings()

**CrewAI Agent Configuration**

This section defines specialized AI agents that drive the lesson planning and generation process. The Curriculum Planner agent is tasked with breaking down any topic into logical, step-by-step subtopics, while the Lesson Writer agent crafts clear, engaging lesson content with explanations and analogies. Together, these agents form the collaborative intelligence at the core of the AI Learning Tutor.

In [28]:
# Define CrewAI Agents

lesson_planner = Agent(
    role="Curriculum Planner",
    goal="Break down any topic into logical subtopics",
    backstory="You're a professional curriculum designer who can deconstruct complex topics into step-by-step lessons.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

lesson_generator = Agent(
    role="Lesson Writer",
    goal="Create lessons with explanations and analogies",
    backstory="You're an educator who writes clear, engaging, and structured lesson content for self-learners.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

**Tutor Logic Overview**

The Tutor Logic function manages the core flow of lesson creation. Once a topic and learner difficulty are selected, it directs CrewAI agents to break down the subject, generate a tailored lesson, and create focused flashcards, all designed to match the learner’s skill level. This ensures a cohesive, adaptive, and user-centered learning experience from start to finish.

In [29]:
# Tutor Logic

def run_tutor(topic, difficulty):
    try:
        # Use difficulty level in prompt
        planning_task = Task(
            description=(
                f"Break down the topic: {topic} into a logical series of subtopics for a {difficulty.lower()} learner "
                "that build understanding step by step."
            ),
            agent=lesson_planner,
            expected_output=(
                "A list of clear and logically ordered subtopics required to learn the full topic."
            )
        )

        generation_task = Task(
            description=(
                f"Write a structured and engaging {difficulty.lower()}-level lesson based on the first subtopic of the topic: {topic}.\n"
                "Include the following clearly labeled sections:\n"
                "1. Title (formatted as 'Lesson: ...')\n"
                "2. Definition\n"
                "3. Explanation with Examples\n"
                "4. Analogies\n"
                "5. Summary\n\n"
                "Write in full sentences. Avoid bullet points or markdown."
            ),
            agent=lesson_generator,
            expected_output="A complete educational lesson."
        )

        flashcard_task = Task(
            description=(
                f"Generate 3 flashcards for the topic: {topic}, tailored to a {difficulty.lower()} learner.\n"
                "Format:\n"
                "Q1: ...\nA1: ...\nQ2: ...\nA2: ...\nQ3: ...\nA3: ...\n\n"
                "Plain text only. No markdown or lists."
            ),
            agent=lesson_generator,
            expected_output="Three clearly numbered flashcards in Q&A format."
        )

        crew = Crew(
            agents=[lesson_planner, lesson_generator],
            tasks=[planning_task, generation_task, flashcard_task],
            verbose=True
        )
        crew.kickoff()

        lesson = str(generation_task.output).strip() if generation_task.output else " Lesson not available."
        flashcards = str(flashcard_task.output).strip() if flashcard_task.output else " Flashcards not available."

        return lesson, flashcards

    except Exception as e:
        return f" Error: {str(e)}", ""


**User Interface and Interaction**

Leveraging Gradio’s intuitive framework, this interface invites users to engage directly with the AI tutor. The layout features straightforward inputs for topic selection and difficulty, alongside immediate outputs for the generated lesson and flashcards. This interactive setup streamlines the learning process, delivering personalized educational content in a user-friendly format.

In [33]:
import gradio as gr

def tutor_interface(topic, difficulty):
    lesson, flashcards = run_tutor(topic, difficulty)
    return lesson, flashcards

with gr.Blocks(title="AI Tutor") as demo:
    # Header and Description
    gr.Markdown("## 🧠 AI Tutor")
    gr.Markdown("Enter a topic to generate a structured lesson and helpful flashcards using CrewAI + OpenAI.")


    # Topic Input
    with gr.Row():
        topic_input = gr.Textbox(label="Enter Topic", placeholder="e.g., Machine Learning")
        difficulty_input = gr.Dropdown(
            choices=["Beginner", "Intermediate", "Advanced"],
            label="🎯 Select Difficulty Level",
            value="Beginner"
        )
    # Submit Button
    submit_btn = gr.Button("🚀 Submit")

    # Output Row
    with gr.Row():
        lesson_output = gr.Textbox(label="📘 Lesson", lines=20)
        flashcard_output = gr.Textbox(label="🧠 Flashcards", lines=20)

    # Click Logic
    submit_btn.click(fn=tutor_interface, inputs=[topic_input, difficulty_input], outputs=[lesson_output, flashcard_output])

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c8801d48fac16c7e8c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
